https://neo4j.com/labs/genai-ecosystem/graphrag-python/

https://neo4j.com/docs/neo4j-graphrag-python/current/user_guide_rag.html#

In [13]:
import sys
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler(sys.stdout))

In [5]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "anatomy-exodus-pigment-colombo-octopus-1216"

In [6]:
# Neo4j Driver
import neo4j

neo4j_driver = neo4j.GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

# LLM and Embedding Model
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

llm = OpenAILLM(
    model_name="gpt-4o",
    model_params={
        "temperature": 0  # turning temperature down for more deterministic results
    }
)

embedder = OpenAIEmbeddings()

In [14]:
# Vector Retriever
from neo4j_graphrag.generation import GraphRAG, RagTemplate
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder
)

vector_rag = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=RagTemplate())

temp = vector_rag.search("넷플릭스 요금제에 대해 알려줘", retriever_config={'top_k': 5}, return_context=True)
temp

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RagResultModel(answer='죄송하지만, 제공된 문맥에는 넷플릭스 요금제에 대한 정보가 포함되어 있지 않습니다. 대신, 다른 서비스 요금제에 대한 정보가 포함되어 있습니다. 예를 들어, "V컬러링 바이브 플러스"는 월 8,800원에 제공되며, "티빙 베이직"은 월 9,500원에 제공됩니다. 넷플릭스 요금제에 대한 정보를 원하시면, 넷플릭스 공식 웹사이트를 참조하시기 바랍니다.', retriever_result=RetrieverResult(items=[RetrieverResultItem(content='{\'id\': \'65c346e26a0d167f88329fc4e8fea89c\', \'text\': \'{\\n  "요금내용": "월 8,800원",\\n  "서비스명": "V컬러링 바이브 플러스",\\n  "카테고리코드": "M10053",\\n  "사용여부": "Y",\\n  "ALIAS": "per_mo_service",\\n  "상세주소": "/mobile/plan/addon/addon-media/LRZ0003130",\\n  "서비스설명": "V컬러링과 NAVER VIBE 앱 무제한 듣기를  할인된 금액으로 이용 할 수 있는 서비스",\\n  "서비스코드": "LRZ0003130",\\n  "카테고리명": "영상/음악",\\n  "부가가치세": "부가세 포함 금액",\\n  "구분값": "LRZ0003130",\\n  "IOS이미지": "",\\n  "text": "카테고리명:영상/음악 부가가치세:부가세 포함 금액 서비스코드:LRZ0003130 카테고리코드:M10053 서비스명:V컬러링 바이브 플러스 ALIAS:per_mo_service 요금내용:월 8,800원 서비스설명:V컬러링과 NAVER VIBE 앱 무제한 듣기를  할인된 금액으로 이용 할 수 있는 서비스 IOS이미지: 상세주소:/mobile/plan/addon/addon-media/LRZ0003130 구분값:LRZ0003130 사용여부:Y 안드로이드이미지:",\\n  "안드로이드이미

In [20]:
from neo4j_graphrag.generation import GraphRAG, RagTemplate
from neo4j_graphrag.retrievers import VectorCypherRetriever

graph_retriever = VectorCypherRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query="""
MATCH (node)-[relList]-{1, 2}(nb: !Document)
RETURN nb.id, relList[0]
"""
)

graph_rag = GraphRAG(llm=llm, retriever=graph_retriever, prompt_template=RagTemplate())

result = graph_rag.search("넷플릭스 요금제에 대해 알려줘", retriever_config={'top_k': 1}, return_context=True)
result

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RagResultModel(answer='넷플릭스 요금제는 다양한 옵션을 제공하며, 사용자의 필요에 따라 선택할 수 있습니다. 일반적으로 넷플릭스는 기본, 스탠다드, 프리미엄 요금제를 제공하며, 각 요금제는 스트리밍 화질, 동시 시청 가능 기기 수 등의 차이가 있습니다. 기본 요금제는 SD 화질로 1개의 기기에서 시청할 수 있으며, 스탠다드 요금제는 HD 화질로 2개의 기기에서 동시 시청이 가능합니다. 프리미엄 요금제는 UHD 화질로 최대 4개의 기기에서 동시 시청이 가능합니다. 요금제에 대한 자세한 정보는 넷플릭스 공식 웹사이트에서 확인할 수 있습니다.', retriever_result=RetrieverResult(items=[RetrieverResultItem(content="<Record nb.id='Y' relList[0]=<Relationship element_id='5:e91334ed-cf36-4fa8-a84c-5afca4e086d3:1161928703861589653' nodes=(<Node element_id='4:e91334ed-cf36-4fa8-a84c-5afca4e086d3:1685' labels=frozenset() properties={}>, <Node element_id='4:e91334ed-cf36-4fa8-a84c-5afca4e086d3:29' labels=frozenset() properties={}>) type='MENTIONS' properties={}>>", metadata=None), RetrieverResultItem(content="<Record nb.id='유쓰 5G 다이렉트 46' relList[0]=<Relationship element_id='5:e91334ed-cf36-4fa8-a84c-5afca4e086d3:1161928703861589653' nodes=(<Node element_id='4:e91334ed-cf36-4fa8-a84c-5afca4e086d3:1685' labels=frozenset() prop

In [4]:
from langchain_neo4j import Neo4jGraph

enhanced_graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    enhanced_schema=True
)

In [10]:
# Text2Cypher Retriever
from neo4j_graphrag.retrievers import Text2CypherRetriever
from neo4j_graphrag.generation import GraphRAG, RagTemplate

text2cypher_retriever = Text2CypherRetriever(
    driver=neo4j_driver,
    llm=llm,
    neo4j_schema=enhanced_graph.schema
)

text2cypher_rag = GraphRAG(
    llm=llm,
    retriever=text2cypher_retriever,
    prompt_template=RagTemplate(
        system_instructions="""You are expert of QA. Answer the question with exact information.
        DO NOT use information like boolean or 'Y','N'.
        Domain url is 'https://www.lguplus.com/'.
        """
    )
)

result = text2cypher_rag.search(
    "",
    return_context=True
)
result

RagResultModel(answer="죄송하지만, 'https://www.lguplus.com/' 도메인에서 넷플릭스 요금제에 대한 구체적인 정보를 제공할 수 없습니다. 해당 웹사이트를 직접 방문하여 최신 요금제 정보를 확인하시기 바랍니다.", retriever_result=RetrieverResult(items=[], metadata={'cypher': "cypher\nMATCH (p:Product)-[:HAS_DESCRIPTION]->(d:Description)\nWHERE d.id CONTAINS '넷플릭스'\nRETURN p\n", '__retriever': 'Text2CypherRetriever'}))